In [1]:
print("all ok")

all ok


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader

loader=DirectoryLoader(path='F:\\Tapas\\Learning\\GenAI\\data')

In [3]:
doc=loader.load()

c:\Users\Hp\miniconda3\envs\genai_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [4]:
spliter=RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=64)

In [5]:
doc

[Document(metadata={'source': 'F:\\Tapas\\Learning\\GenAI\\data\\2501.09136v3.pdf'}, page_content='5 2 0 2\n\nb e F 4\n\n] I\n\nA . s c [\n\n3 v 6 3 1 9 0 . 1 0 5 2 : v i X r a\n\nAGENTIC RETRIEVAL-AUGMENTED GENERATION: A SURVEY ON AGENTIC RAG\n\nAditi Singh Department of Computer Science Cleveland State University Cleveland, OH, USA a.singh22@csuohio.edu\n\nAbul Ehtesham The Davey Tree Expert Company Kent, OH, USA abul.ehtesham@davey.com\n\nSaket Kumar The MathWorks Inc Natick, MA, USA saketk@mathworks.com\n\nTala Talaei Khoei Khoury College of Computer Science Roux Institute at Northeastern University Portland, ME, USA t.talaeikhoei@northeastern.edu\n\nABSTRACT\n\nLarge Language Models (LLMs) have revolutionized artificial intelligence (AI) by enabling human- like text generation and natural language understanding. However, their reliance on static training data limits their ability to respond to dynamic, real-time queries, resulting in outdated or inaccurate outputs. Retrieval-Augme

In [6]:
docs=spliter.split_documents(doc)

In [7]:
len(docs)

387

In [14]:
from langchain_openai.embeddings import OpenAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")
embeddings=OpenAIEmbeddings()

len(embeddings.embed_query("hello how are you?"))

1536

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
ASTRA_DB_API_ENDPOINT="load from env"
ASTRA_DB_APPLICATION_TOKEN="load from env file"
ASTRA_DB_KEYSPACE="load from env file"

In [ ]:
from langchain_astradb import AstraDBVectorStore


In [16]:
vector_store=AstraDBVectorStore(
    collection_name="multidoc_vector",
    embedding=embeddings,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE
)

In [17]:
inserted_ids=vector_store.add_documents(docs)

In [18]:
print(f"Inserted {len(inserted_ids)} records")

Inserted 387 records


In [19]:
retriever=vector_store.as_retriever(search_kwargs={"k":5})

In [20]:
retriever

VectorStoreRetriever(tags=['AstraDBVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_astradb.vectorstores.AstraDBVectorStore object at 0x00000255D8D78290>, search_kwargs={'k': 5})

In [21]:
prompt_template="""
You are an AI philosopher drawing insights from the roadmap of the data. craft a thoughtful 
response based on the provided insights. your response should be consise and strictly
related to the provided context and user query.

If the context does not cosist any valid response then say you don't know.

Context:
{context}

Question:
{question}
Your Response:
"""


In [22]:
from langchain_core.prompts import ChatPromptTemplate

In [23]:
prompt=ChatPromptTemplate.from_template(template=prompt_template)

In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [32]:
rag_chain= (
    {"context" : retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
) 

In [33]:
rag_chain.invoke("What is Modular RAG?")

'Modular RAG is the latest evolution in RAG paradigms, emphasizing flexibility and customization. It decomposes the retrieval and generation pipeline into independent, reusable components, enabling domain-specific optimization and task adaptability, making it ideal for complex, multi-domain tasks requiring scalability and precision.'

In [34]:
rag_chain.invoke("what is the USA stand on Ukraine?")

"The United States stands with the Ukrainian people, condemning Russia's unprovoked aggression and believing that dictators must pay a price for their actions to prevent further chaos. While US forces will not engage directly in conflict with Russian forces in Ukraine, the US, as a NATO member, will defend every inch of NATO territory with its full collective power, mobilizing forces to protect allies. American diplomacy and resolve are committed to imposing a continuing high price on Russia over the long run."

In [35]:
rag_chain.invoke("what is Generative AI?")

'Generative AI is a subset of artificial intelligence designed to create new content, such as text, images, or music. It leverages sophisticated algorithms and deep learning models, including neural networks, to produce outputs that resemble those created by humans by learning from existing data.'